# Captstone Project for submission

In [137]:
import requests
import lxml.html as lh
import pandas as pd
import numpy as np

Scrapping Data

In [138]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [139]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [140]:
# Parse the first row as our header
tr_elements = doc.xpath('//tr')

#Create empty list
col=[]
i=0

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postal Code
"
2:"Borough
"
3:"Neighborhood
"


In [141]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [142]:
# Check the length of each column. Ideally, they should all be the same
[len(C) for (title,C) in col]

[181, 181, 181]

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [143]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [144]:
# Access the top 5 rows of the data frame 
df.head()

,Postal Code\n,Borough\n,Neighborhood\n
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [145]:
df = df.replace('\n',' ', regex=True)
df.head()

,Postal Code\n,Borough\n,Neighborhood\n
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [146]:
df.rename(columns={'Postal Code\n': 'Postal Code', 'Borough\n': 'Borough','Neighborhood\n':'Neighborhood'}, inplace=True)


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [147]:
df.drop(df.index[df['Borough'] == 'Not assigned '], inplace = True)

# Reset the index and dropping the previous index
df = df.reset_index(drop=True)

df.head(10)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [151]:
df = df.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(','.join).reset_index()
df.columns = ['Postal Code','Borough','Neighborhood']

df.drop([0],axis=0,inplace=True)
df.head(10)

,Postal Code,Borough,Neighborhood
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae
6,M1J,Scarborough,Scarborough Village
7,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
8,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
9,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
10,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [152]:
df['Neighborhood'] = df['Neighborhood'].str.strip()


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [153]:
df.loc[df['Neighborhood'] == 'Not assigned ', 'Neighborhood'] = df['Borough']

In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [154]:
df.shape

(103, 3)

In [155]:
df.to_csv(r'toronto.csv')